In [8]:
import pandas as pd

# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------


In [9]:
import mysql.connector
from mysql.connector import Error
# Función para conectar con MySQL y crear la base de datos y tablas
def create_database_and_tables():
    try:
        # Conectar al servidor de MySQL (sin base de datos específica, para poder crear una nueva)
        connection = mysql.connector.connect(
            host='127.0.0.1',        # Cambia esto si tu servidor no está local
            user='root',             # Tu usuario de MySQL (usualmente 'root' en local)
            password='AlumnaAdalab' # Tu contraseña de MySQL
        )
        if connection.is_connected():
            print("Conexión exitosa al servidor MySQL")
            # Crear un cursor para ejecutar sentencias SQL
            cursor = connection.cursor()
            # Crear la base de datos
            cursor.execute("CREATE DATABASE IF NOT EXISTS employee_db")
            print("Base de datos 'employee_db' creada o ya existe.")
            # Seleccionar la base de datos recién creada
            cursor.execute("USE employee_db")
            # Crear las tablas
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS employees (
                employee_number INT PRIMARY KEY,
                age INT,
                gender VARCHAR(10),
                education VARCHAR(50),
                education_field VARCHAR(50),
                job_role VARCHAR(50),
                job_level INT,
                marital_status VARCHAR(50),
                monthly_income DECIMAL(10, 2),
                salary DECIMAL(10, 2),
                remote_work VARCHAR(10)
            );
            """)
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS performance_and_satisfaction (
                employee_number INT,
                job_satisfaction INT,
                performance_rating INT,
                environment_satisfaction INT,
                relationship_satisfaction INT,
                work_life_balance INT,
                FOREIGN KEY (employee_number) REFERENCES employees(employee_number)
            );
            """)
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS employment_and_experience (
                employee_number INT,
                years_at_company INT,
                years_since_last_promotion INT,
                years_with_curr_manager INT,
                total_working_years INT,
                num_companies_worked INT,
                FOREIGN KEY (employee_number) REFERENCES employees(employee_number)
            );
            """)
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS work_conditions (
                employee_number INT,
                daily_rate DECIMAL(10, 2),
                distance_from_home DECIMAL(10, 2),
                overtime VARCHAR(10),
                standard_hours INT,
                FOREIGN KEY (employee_number) REFERENCES employees(employee_number)
            );
            """)
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS business_travel (
                employee_number INT,
                business_travel VARCHAR(50),
                FOREIGN KEY (employee_number) REFERENCES employees(employee_number)
            );
            """)
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS salary_hike (
                employee_number INT,
                percent_salary_hike DECIMAL(5, 2),
                FOREIGN KEY (employee_number) REFERENCES employees(employee_number)
            );
            """)
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS job_involvement (
                employee_number INT,
                job_involvement INT,
                FOREIGN KEY (employee_number) REFERENCES employees(employee_number)
            );
            """)
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS company_and_stock_options (
                employee_number INT,
                num_companies_worked INT,
                stock_option_level INT,
                FOREIGN KEY (employee_number) REFERENCES employees(employee_number)
            );
            """)
            print("Todas las tablas han sido creadas exitosamente.")
    except Error as e:
        print("Error al conectar con MySQL:", e)
    finally:
        if connection.is_connected():
            # Cerrar la conexión
            cursor.close()
            connection.close()
            print("Conexión cerrada.")
# Llamada a la función
create_database_and_tables()

Conexión exitosa al servidor MySQL
Base de datos 'employee_db' creada o ya existe.
Todas las tablas han sido creadas exitosamente.
Conexión cerrada.


In [10]:
df_final= pd.read_csv("df_final.csv") 
df_final.head()

,Unnamed: 0,age,attrition,business_travel,daily_rate,distance_from_home,education,education_field,employee_number,environment_satisfaction,...,standard_hours,stock_option_level,total_working_years,training_times_last_year,work_life_balance,years_at_company,years_since_last_promotion,years_with_curr_manager,salary,remote_work
0,0,51,No,Unknown,2015.722222,6,3,Unknown,1,1,...,Full Time,0,Unknown,5,3.0,20,15,15,195370.00,Y
1,1,52,No,Unknown,2063.388889,1,4,life sciences,2,3,...,Unknown,1,34.0,5,3.0,33,11,9,199990.00,Y
2,2,42,No,travel_rarely,1984.253968,4,2,technical degree,3,3,...,Unknown,0,22.0,3,Unknown,22,11,15,192320.00,Y
3,3,47,No,travel_rarely,1771.404762,2,4,medical,4,1,...,Full Time,2,Unknown,2,Unknown,20,5,6,171690.00,N
4,4,46,No,Unknown,1582.771346,3,3,technical degree,5,1,...,Unknown,1,Unknown,5,3.0,19,2,8,153407.04,N


In [11]:
df_final.columns

Index(['Unnamed: 0', 'age', 'attrition', 'business_travel', 'daily_rate',
       'distance_from_home', 'education', 'education_field', 'employee_number',
       'environment_satisfaction', 'gender', 'job_involvement', 'job_level',
       'job_role', 'job_satisfaction', 'marital_status', 'monthly_income',
       'monthly_rate', 'num_companies_worked', 'overtime',
       'percent_salary_hike', 'performance_rating',
       'relationship_satisfaction', 'standard_hours', 'stock_option_level',
       'total_working_years', 'training_times_last_year', 'work_life_balance',
       'years_at_company', 'years_since_last_promotion',
       'years_with_curr_manager', 'salary', 'remote_work'],
      dtype='object')

In [14]:
# Creamos la función 'cargar datos' para insertar los datos en las tablas de la base de datos

def cargar_datos (user, password, host, bbdd, query, lista_tuplas):
    cnx = mysql.connector.connect(user=user, password=password, host=host, database=bbdd)
    mycursor = cnx.cursor()
    try:
          mycursor = cnx.cursor()
          mycursor.executemany(query, datos_tabla_employees)
          cnx.commit()
          print(mycursor.rowcount, "row/s inserted successfully")
          cnx.close()
    except mysql.connector.Error as err:
          print(err)
          print("Error Code:", err.errno)
          print("SQLSTATE", err.sqlstate)
          print("Message", err.msg)
          cnx.close()

In [15]:
df_employees = df_final[['employee_number', 'age', 'gender', 'education', 
                         'education_field', 'job_role', 'job_level', 
                         'marital_status', 'monthly_income', 'salary', 'remote_work']]

In [16]:
import mysql.connector
from mysql.connector import Error

try:
    # Conexión a la base de datos
    connection = mysql.connector.connect(
        host='127.0.0.1',
        user='root',
        password='AlumnaAdalab',
        database='employee_db'
    )
    
    if connection.is_connected():
        print("Conexión exitosa a la base de datos")
        cursor = connection.cursor()

        # Consulta de inserción
        query = """
            INSERT INTO employees (
                employee_number, age, gender, education, education_field, 
                job_role, job_level, marital_status, monthly_income, salary, remote_work
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
        """

        # Convertir filas del DataFrame a tuplas y ejecutarlas
        rows_as_tuples = [tuple(row) for row in df_employees.itertuples(index=False)]
        
        # Usar executemany para insertar múltiples filas de una vez
        cursor.executemany(query, rows_as_tuples)
        connection.commit()
        
        print(f"Se insertaron {cursor.rowcount} filas correctamente.")
except Error as e:
    print("Error al conectar o insertar en la base de datos:", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("Conexión cerrada.")

Conexión exitosa a la base de datos
Se insertaron 1614 filas correctamente.
Conexión cerrada.
